# Module 1: Data Loading from Multiple Sources

**Author:** Chinmay Nadgir  
**Date:** October 2025  
**Purpose:** Demonstrate professional data loading techniques from CSV, Excel, JSON, and text files

---

## Table of Contents
1. [Introduction](#intro)
2. [Setup & Imports](#setup)
3. [Loading CSV Files](#csv)
4. [Loading Excel Files](#excel)
5. [Loading JSON Files](#json)
6. [Loading Text Files](#text)
7. [Error Handling](#errors)
8. [Data Format Comparison](#comparison)
9. [Saving to Pickle](#pickle)
10. [Summary](#summary)

<a id='intro'></a>
## 1. Introduction

This notebook demonstrates best practices for loading data from various file formats into pandas DataFrames. Understanding these techniques is crucial for any data science workflow.

**Learning Objectives:**
- Load data from CSV, Excel, JSON, and text files
- Handle common data loading issues (encoding, delimiters, missing files)
- Compare performance and use cases for different formats
- Implement robust error handling
- Save processed data efficiently

<a id='setup'></a>
## 2. Setup & Imports

In [3]:
# Standard library imports
import os
import warnings
from pathlib import Path
from typing import Optional
# Third-party imports
import pandas as pd
import numpy as np

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 2)

# Display versions
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

pandas version: 2.3.3
numpy version: 2.3.3


In [4]:
def display_dataframe_info(df: pd.DataFrame, name: str) -> None:
    """
    Display comprehensive information about a DataFrame.
    
    Parameters:
    -----------
    df : pd.DataFrame
        The DataFrame to analyze
    name : str
        Name of the dataset for display purposes
    """
    print(f"\n{'='*60}")
    print(f"Dataset: {name}")
    print(f"{'='*60}")
    print(f"\nShape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"\nData Types:")
    print(df.dtypes.value_counts())
    print(f"\nFirst 5 Rows:")
    display(df.head())
    print(f"\nBasic Info:")
    print(df.info())
    print(f"\nMissing Values:")
    missing = df.isnull().sum()
    if missing.sum() > 0:
        missing_pct = (missing / len(df) * 100).round(2)
        missing_df = pd.DataFrame({
            'Missing_Count': missing[missing > 0],
            'Percentage': missing_pct[missing > 0]
        })
        display(missing_df)
    else:
        print("No missing values found.")

### Create Sample Datasets

For demonstration purposes, we'll create sample datasets in different formats.

In [5]:
# Create data directory if it doesn't exist
data_dir = Path('data')
data_dir.mkdir(exist_ok=True)

# Create sample dataset
np.random.seed(42)
sample_data = pd.DataFrame({
    'customer_id': range(1, 101),
    'name': [f'Customer_{i}' for i in range(1, 101)],
    'age': np.random.randint(18, 70, 100),
    'purchase_amount': np.random.uniform(10, 1000, 100).round(2),
    'category': np.random.choice(['Electronics', 'Clothing', 'Food', 'Books'], 100),
    'date': pd.date_range('2024-01-01', periods=100, freq='D'),
    'loyalty_member': np.random.choice([True, False], 100)
})

# Introduce some missing values for realism
sample_data.loc[5:8, 'age'] = np.nan
sample_data.loc[15:17, 'purchase_amount'] = np.nan

print("Sample dataset created successfully!")
print(f"Shape: {sample_data.shape}")

Sample dataset created successfully!
Shape: (100, 7)


<a id='csv'></a>
## 3. Loading CSV Files

CSV (Comma-Separated Values) is the most common format for tabular data.

**Key Parameters:**
- `sep`: Delimiter (default: ',')
- `encoding`: File encoding (default: 'utf-8')
- `parse_dates`: Columns to parse as dates
- `na_values`: Additional values to recognize as NaN
- `dtype`: Specify data types for columns

In [6]:
# Save sample data as CSV
csv_path = data_dir / 'sample_data.csv'
sample_data.to_csv(csv_path, index=False, encoding='utf-8')
print(f"CSV file saved to: {csv_path}")

CSV file saved to: data/sample_data.csv


In [7]:
# Load CSV file with proper handling
df_csv = pd.read_csv(
    csv_path,
    encoding='utf-8',
    parse_dates=['date'],
    dtype={
        'customer_id': 'int64',
        'name': 'string',
        'category': 'category',
        'loyalty_member': 'bool'
    }
)

display_dataframe_info(df_csv, 'CSV Dataset')


Dataset: CSV Dataset

Shape: 100 rows × 7 columns
Memory Usage: 0.01 MB

Data Types:
float64           2
int64             1
string[python]    1
category          1
datetime64[ns]    1
bool              1
Name: count, dtype: int64

First 5 Rows:


,customer_id,name,age,purchase_amount,category,date,loyalty_member
0,1,Customer_1,56.0,173.61,Books,2024-01-01,False
1,2,Customer_2,69.0,25.48,Electronics,2024-01-02,False
2,3,Customer_3,46.0,429.17,Food,2024-01-03,False
3,4,Customer_4,32.0,400.93,Books,2024-01-04,True
4,5,Customer_5,60.0,300.55,Books,2024-01-05,True



Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   customer_id      100 non-null    int64         
 1   name             100 non-null    string        
 2   age              96 non-null     float64       
 3   purchase_amount  97 non-null     float64       
 4   category         100 non-null    category      
 5   date             100 non-null    datetime64[ns]
 6   loyalty_member   100 non-null    bool          
dtypes: bool(1), category(1), datetime64[ns](1), float64(2), int64(1), string(1)
memory usage: 4.4 KB
None

Missing Values:


,Missing_Count,Percentage
age,4,4.0
purchase_amount,3,3.0


<a id='excel'></a>
## 4. Loading Excel Files

Excel files (.xlsx, .xls) can contain multiple sheets and formatting.

**Key Parameters:**
- `sheet_name`: Specify which sheet to load (default: 0)
- `engine`: Parser engine ('openpyxl' for .xlsx, 'xlrd' for .xls)
- `usecols`: Specify columns to load
- `skiprows`: Rows to skip at the beginning

In [10]:
# Save sample data as Excel
excel_path = data_dir / 'sample_data.xlsx'
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    sample_data.to_excel(writer, sheet_name='Customers', index=False)
    # Create a second sheet with summary statistics
    summary = sample_data.describe()
    summary.to_excel(writer, sheet_name='Summary')
    
print(f"Excel file saved to: {excel_path}")

Excel file saved to: data/sample_data.xlsx


In [11]:
# Load Excel file - single sheet
df_excel = pd.read_excel(
    excel_path,
    sheet_name='Customers',
    engine='openpyxl',
    parse_dates=['date']
)

display_dataframe_info(df_excel, 'Excel Dataset (Customers Sheet)')


Dataset: Excel Dataset (Customers Sheet)

Shape: 100 rows × 7 columns
Memory Usage: 0.01 MB

Data Types:
object            2
float64           2
int64             1
datetime64[ns]    1
bool              1
Name: count, dtype: int64

First 5 Rows:


,customer_id,name,age,purchase_amount,category,date,loyalty_member
0,1,Customer_1,56.0,173.61,Books,2024-01-01,False
1,2,Customer_2,69.0,25.48,Electronics,2024-01-02,False
2,3,Customer_3,46.0,429.17,Food,2024-01-03,False
3,4,Customer_4,32.0,400.93,Books,2024-01-04,True
4,5,Customer_5,60.0,300.55,Books,2024-01-05,True



Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   customer_id      100 non-null    int64         
 1   name             100 non-null    object        
 2   age              96 non-null     float64       
 3   purchase_amount  97 non-null     float64       
 4   category         100 non-null    object        
 5   date             100 non-null    datetime64[ns]
 6   loyalty_member   100 non-null    bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 4.9+ KB
None

Missing Values:


,Missing_Count,Percentage
age,4,4.0
purchase_amount,3,3.0


In [12]:
# Load multiple sheets
excel_dict = pd.read_excel(
    excel_path,
    sheet_name=None,  # Load all sheets
    engine='openpyxl'
)

print(f"\nNumber of sheets: {len(excel_dict)}")
print(f"Sheet names: {list(excel_dict.keys())}")
print(f"\nSummary Sheet Preview:")
display(excel_dict['Summary'].head())


Number of sheets: 2
Sheet names: ['Customers', 'Summary']

Summary Sheet Preview:


,Unnamed: 0,customer_id,age,purchase_amount,date
0,count,100.00,96.00,97.00,100
1,mean,50.50,43.54,478.38,2024-02-19 12:00:00
2,min,1.00,19.00,10.52,2024-01-01 00:00:00
3,25%,25.75,31.75,254.41,2024-01-25 18:00:00
4,50%,50.50,42.00,454.96,2024-02-19 12:00:00


<a id='json'></a>
## 5. Loading JSON Files

JSON (JavaScript Object Notation) is common for nested/hierarchical data.

**Key Parameters:**
- `orient`: Format of JSON string ('records', 'index', 'columns', 'values', 'split', 'table')
- `lines`: Read file as line-delimited JSON
- `dtype`: Specify data types
- `convert_dates`: Parse dates automatically

In [13]:
# Save sample data as JSON (records orientation)
json_path = data_dir / 'sample_data.json'
sample_data.to_json(json_path, orient='records', date_format='iso', indent=2)
print(f"JSON file saved to: {json_path}")

JSON file saved to: data/sample_data.json


In [14]:
# Load JSON file
df_json = pd.read_json(
    json_path,
    orient='records',
    convert_dates=['date']
)

display_dataframe_info(df_json, 'JSON Dataset')


Dataset: JSON Dataset

Shape: 100 rows × 7 columns
Memory Usage: 0.01 MB

Data Types:
object            2
float64           2
int64             1
datetime64[ns]    1
bool              1
Name: count, dtype: int64

First 5 Rows:


,customer_id,name,age,purchase_amount,category,date,loyalty_member
0,1,Customer_1,56.0,173.61,Books,2024-01-01,False
1,2,Customer_2,69.0,25.48,Electronics,2024-01-02,False
2,3,Customer_3,46.0,429.17,Food,2024-01-03,False
3,4,Customer_4,32.0,400.93,Books,2024-01-04,True
4,5,Customer_5,60.0,300.55,Books,2024-01-05,True



Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   customer_id      100 non-null    int64         
 1   name             100 non-null    object        
 2   age              96 non-null     float64       
 3   purchase_amount  97 non-null     float64       
 4   category         100 non-null    object        
 5   date             100 non-null    datetime64[ns]
 6   loyalty_member   100 non-null    bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 4.9+ KB
None

Missing Values:


,Missing_Count,Percentage
age,4,4.0
purchase_amount,3,3.0


<a id='text'></a>
## 6. Loading Text Files with Custom Delimiters

Text files may use various delimiters (tab, pipe, semicolon, etc.).

**Common Use Cases:**
- Tab-separated values (TSV): `sep='\t'`
- Pipe-delimited: `sep='|'`
- Fixed-width files: Use `pd.read_fwf()`

In [15]:
# Save sample data as tab-delimited text file
txt_path = data_dir / 'sample_data.txt'
sample_data.to_csv(txt_path, sep='\t', index=False, encoding='utf-8')
print(f"Text file saved to: {txt_path}")

Text file saved to: data/sample_data.txt


In [16]:
# Load tab-delimited text file
df_txt = pd.read_csv(
    txt_path,
    sep='\t',
    encoding='utf-8',
    parse_dates=['date']
)

display_dataframe_info(df_txt, 'Text File Dataset')


Dataset: Text File Dataset

Shape: 100 rows × 7 columns
Memory Usage: 0.01 MB

Data Types:
object            2
float64           2
int64             1
datetime64[ns]    1
bool              1
Name: count, dtype: int64

First 5 Rows:


,customer_id,name,age,purchase_amount,category,date,loyalty_member
0,1,Customer_1,56.0,173.61,Books,2024-01-01,False
1,2,Customer_2,69.0,25.48,Electronics,2024-01-02,False
2,3,Customer_3,46.0,429.17,Food,2024-01-03,False
3,4,Customer_4,32.0,400.93,Books,2024-01-04,True
4,5,Customer_5,60.0,300.55,Books,2024-01-05,True



Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   customer_id      100 non-null    int64         
 1   name             100 non-null    object        
 2   age              96 non-null     float64       
 3   purchase_amount  97 non-null     float64       
 4   category         100 non-null    object        
 5   date             100 non-null    datetime64[ns]
 6   loyalty_member   100 non-null    bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 4.9+ KB
None

Missing Values:


,Missing_Count,Percentage
age,4,4.0
purchase_amount,3,3.0


<a id='errors'></a>
## 7. Error Handling & Robust Loading

Production code should handle common errors gracefully.

In [17]:
def load_data_safely(file_path: str, file_type: str = 'csv') -> Optional[pd.DataFrame]:
    """
    Safely load data from various file formats with error handling.
    
    Parameters:
    -----------
    file_path : str
        Path to the file
    file_type : str
        Type of file ('csv', 'excel', 'json', 'txt')
    
    Returns:
    --------
    pd.DataFrame or None
        Loaded DataFrame, or None if loading failed
    """
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        
        if file_type == 'csv':
            df = pd.read_csv(file_path, encoding='utf-8')
        elif file_type == 'excel':
            df = pd.read_excel(file_path, engine='openpyxl')
        elif file_type == 'json':
            df = pd.read_json(file_path, orient='records')
        elif file_type == 'txt':
            df = pd.read_csv(file_path, sep='\t', encoding='utf-8')
        else:
            raise ValueError(f"Unsupported file type: {file_type}")
        
        print(f"✓ Successfully loaded {file_type.upper()} file: {file_path}")
        print(f"  Shape: {df.shape}")
        return df
    
    except FileNotFoundError as e:
        print(f"✗ Error: {e}")
        return None
    except pd.errors.EmptyDataError:
        print(f"✗ Error: The file is empty: {file_path}")
        return None
    except pd.errors.ParserError as e:
        print(f"✗ Parsing Error: {e}")
        return None
    except Exception as e:
        print(f"✗ Unexpected Error: {type(e).__name__}: {e}")
        return None

In [18]:
# Test the safe loading function
print("Testing safe data loading function:\n")

# Test with existing file
df_test1 = load_data_safely(str(csv_path), 'csv')

# Test with non-existent file
df_test2 = load_data_safely('nonexistent_file.csv', 'csv')

# Test with unsupported file type
df_test3 = load_data_safely(str(csv_path), 'xml')

Testing safe data loading function:

✓ Successfully loaded CSV file: data/sample_data.csv
  Shape: (100, 7)
✗ Error: File not found: nonexistent_file.csv
✗ Unexpected Error: ValueError: Unsupported file type: xml


<a id='comparison'></a>
## 8. Data Format Comparison

Each format has trade-offs in terms of speed, size, and compatibility.

In [19]:
import time

# Measure file sizes
file_sizes = {
    'CSV': os.path.getsize(csv_path) / 1024,  # KB
    'Excel': os.path.getsize(excel_path) / 1024,
    'JSON': os.path.getsize(json_path) / 1024,
    'Text': os.path.getsize(txt_path) / 1024
}

# Measure loading times
loading_times = {}

# CSV
start = time.time()
pd.read_csv(csv_path)
loading_times['CSV'] = (time.time() - start) * 1000  # milliseconds

# Excel
start = time.time()
pd.read_excel(excel_path, engine='openpyxl')
loading_times['Excel'] = (time.time() - start) * 1000

# JSON
start = time.time()
pd.read_json(json_path)
loading_times['JSON'] = (time.time() - start) * 1000

# Text
start = time.time()
pd.read_csv(txt_path, sep='\t')
loading_times['Text'] = (time.time() - start) * 1000

# Create comparison table
comparison_df = pd.DataFrame({
    'Format': list(file_sizes.keys()),
    'File_Size_KB': list(file_sizes.values()),
    'Load_Time_ms': list(loading_times.values())
})

comparison_df['File_Size_KB'] = comparison_df['File_Size_KB'].round(2)
comparison_df['Load_Time_ms'] = comparison_df['Load_Time_ms'].round(2)

print("\n" + "="*60)
print("DATA FORMAT PERFORMANCE COMPARISON")
print("="*60)
display(comparison_df)


DATA FORMAT PERFORMANCE COMPARISON


,Format,File_Size_KB,Load_Time_ms
0,CSV,5.00,2.71
1,Excel,9.65,37.86
2,JSON,18.83,8.72
3,Text,5.00,2.23


### Format Recommendations

| Format | Best Use Case | Pros | Cons |
|--------|---------------|------|------|
| **CSV** | General-purpose, data exchange | Universal compatibility, small size, fast | No metadata, single sheet |
| **Excel** | Business reports, multiple sheets | Familiar, formatting, multiple sheets | Larger files, slower loading |
| **JSON** | Nested data, web APIs | Flexible structure, web-friendly | Larger size, slower for tabular data |
| **Text** | Legacy systems, specific delimiters | Simple, customizable | Requires delimiter specification |
| **Pickle** | Python-to-Python, preserving types | Fastest, preserves data types | Python-only, security concerns |

<a id='pickle'></a>
## 9. Saving to Pickle Format

Pickle is Python's native serialization format. It's fastest and preserves data types perfectly.

**When to use:**
- Intermediate processing steps
- Python-to-Python workflows
- Preserving complex data types

**When NOT to use:**
- Sharing with non-Python users
- Long-term storage (version compatibility)
- Untrusted sources (security risk)

In [20]:
# Save to pickle
pickle_path = data_dir / 'sample_data.pkl'
df_csv.to_pickle(pickle_path)
print(f"Pickle file saved to: {pickle_path}")
print(f"File size: {os.path.getsize(pickle_path) / 1024:.2f} KB")

Pickle file saved to: data/sample_data.pkl
File size: 6.37 KB


In [21]:
# Load from pickle
start = time.time()
df_pickle = pd.read_pickle(pickle_path)
pickle_time = (time.time() - start) * 1000

print(f"\nPickle loading time: {pickle_time:.2f} ms")
print(f"Data types preserved: {(df_pickle.dtypes == df_csv.dtypes).all()}")
display(df_pickle.head())


Pickle loading time: 1.84 ms
Data types preserved: True


,customer_id,name,age,purchase_amount,category,date,loyalty_member
0,1,Customer_1,56.0,173.61,Books,2024-01-01,False
1,2,Customer_2,69.0,25.48,Electronics,2024-01-02,False
2,3,Customer_3,46.0,429.17,Food,2024-01-03,False
3,4,Customer_4,32.0,400.93,Books,2024-01-04,True
4,5,Customer_5,60.0,300.55,Books,2024-01-05,True


<a id='summary'></a>
## 10. Summary

### Key Takeaways

1. **CSV** is the most universal and recommended for general data exchange
2. **Excel** is best for business users and reports with multiple sheets
3. **JSON** works well for nested/hierarchical data from APIs
4. **Pickle** is fastest for Python-to-Python workflows
5. Always implement error handling for production code
6. Specify data types explicitly when possible for better performance
7. Parse dates during loading rather than after for efficiency

### Best Practices

✓ Use `encoding='utf-8'` explicitly for text files  
✓ Parse dates during load with `parse_dates` parameter  
✓ Specify `dtype` for better memory efficiency  
✓ Implement try-except blocks for robust error handling  
✓ Document file formats and structures in README files  
✓ Use `.gitignore` to exclude data files from version control  

### Next Steps

Proceed to **Module 2: Data Preprocessing** to clean and prepare these datasets for analysis.

In [22]:
print("\n" + "="*60)
print("MODULE 1 COMPLETE: Data Loading")
print("="*60)
print(f"\nDatasets loaded successfully:")
print(f"  • CSV: {df_csv.shape}")
print(f"  • Excel: {df_excel.shape}")
print(f"  • JSON: {df_json.shape}")
print(f"  • Text: {df_txt.shape}")
print(f"  • Pickle: {df_pickle.shape}")
print(f"\nAll datasets are identical: {df_csv.equals(df_excel) and df_csv.equals(df_json)}")


MODULE 1 COMPLETE: Data Loading

Datasets loaded successfully:
  • CSV: (100, 7)
  • Excel: (100, 7)
  • JSON: (100, 7)
  • Text: (100, 7)
  • Pickle: (100, 7)

All datasets are identical: False
